# 01. 순차 데이터(Sequential Data)와 RNN 기초
### 과목: NLP Sequence Modeling
---

## 1. 순차 데이터란? (시간의 흐름이 있는 데이터)
우리가 다루는 대부분의 데이터는 **순서**가 중요합니다.
- **텍스트**: "나는 밥을 먹는다" vs "먹는다 밥을 나는" (순서가 다르면 의미가 이상해짐)
- **주식**: 어제 가격이 오늘 가격에 영향을 미침.
- **음성**: 소리의 파동이 연속적으로 이어짐.

> ** 핵심**: "앞의 내용을 알아야 뒤의 내용을 이해할 수 있다!" 

## 2. RNN (Recurrent Neural Network): "기억을 가진 신경망"
기존의 신경망(DNN, CNN)은 **"건망증"**이 심했습니다. 입력(사진) 하나를 보고 나면 바로 까먹었죠.
하지만 RNN은 **"메모장(Hidden State)"**을 가지고 있어서, 이전 단계의 정보를 다음 단계로 넘겨줍니다.

### 🧠 동작 원리 (비유)
- **질문**: "이 영화 정말 ..."
- **일반 신경망**: "..." (앞에 무슨 말이 있었는지 모름)
- **RNN**: "'이' -> '영화' -> '정말' 까지 들었으니, 뒤에는 긍정적인 형용사가 올 거야!"

### ⚠️ 치명적인 단점 (기울기 소멸)
- 문장이 너무 길어지면 앞의 내용을 까먹습니다.
- **비유**: "철수가 어제 밥을 먹고 ... (100마디) ... 그래서 **그**는 배가 불렀다." 
- 여기서 '그'가 누구인지 RNN은 기억을 못 할 수도 있습니다. (이걸 해결한 게 LSTM!)

---

## 3. 실습: PyTorch로 RNN 구조 찍어보기
가장 기본이 되는 RNN 블록이 어떻게 생겼는지 코드로 확인해봅시다.

In [ ]:
import torch
import torch.nn as nn

# --- [1] 데이터 준비 (가상의 문장) ---
# 상황: 2개의 문장이 있고, 각 문장은 3개의 단어로 되어 있으며, 각 단어는 4개의 숫자로 표현됨
batch_size = 2     # 문장 2개 (Batch Size)
seq_len = 3        # 단어 3개 (Time Steps)
input_size = 4     # 단어 하나당 특징 4개 (Embedding Size)
hidden_size = 5    # RNN이 기억할 메모리 크기 (Hidden Size)

# 랜덤 데이터 생성 (입력)
x = torch.randn(batch_size, seq_len, input_size)
print(f"입력 데이터 형태 (B, T, F): {x.shape}")

# --- [2] RNN 모델 만들기 ---
# batch_first=True: (배치, 시간, 특징) 순서로 데이터를 받겠다는 뜻 (PyTorch 기본은 시간, 배치, 특징임)
rnn = nn.RNN(input_size, hidden_size, batch_first=True)

# --- [3] 실행 (Forward) ---
output, hidden = rnn(x)

print("\n--- 결과 확인 ---")
print(f"1. 전체 출력 (Output): {output.shape} -> (배치, 시간, 은닉크기)")
print("   >> 각 단어를 읽을 때마다 뱉어낸 생각들")
print(f"2. 최종 기억 (Hidden): {hidden.shape} -> (층개수, 배치, 은닉크기)")
print("   >> 문장을 다 읽고 나서 머릿속에 남은 마지막 요약본")

---
## 4. 실전: IMDB 영화 리뷰 감성 분석
이제 진짜 데이터를 가지고 놀아봅시다.
**"이 영화 진짜 재밌다"** -> **긍정(1)**
**"돈 아깝다"** -> **부정(0)**

이걸 맞추는 모델을 RNN으로 만들어볼 겁니다.

In [ ]:
from tensorflow.keras.datasets import imdb
import torch

# --- [1] 데이터 로드 ---
# num_words=300: 가장 많이 쓰이는 단어 300개만 사용하겠다. (나머지는 무시)
vocab_size = 300
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=vocab_size)

print(f"학습 데이터: {len(X_train)}개, 테스트 데이터: {len(X_test)}개")

# 데이터 줄이기 (실습 속도를 위해)
X_train, y_train = X_train[:15000], y_train[:15000]
X_test, y_test = X_test[:10000], y_test[:10000]

In [ ]:
# 데이터가 어떻게 생겼는지 눈으로 확인해보자
print("--- 첫 번째 리뷰 데이터 (숫자로 변환된 상태) ---")
print(X_train[0][:20], "... (뒤에 더 있음)")
print(f"라벨: {y_train[0]} (1이면 긍정, 0이면 부정)")

# 원래 이건 영어 문장이었음. 예를 들어 1='START', 14='this', 22='film' ...

### ⚠️ 문제 발생: 문장 길이가 다 다르다!
어떤 리뷰는 짧고("노잼"), 어떤 리뷰는 깁니다.
컴퓨터는 행렬(Matrix) 계산을 해야 해서 **길이를 똑같이 맞춰줘야(Padding)** 합니다.

- 짧으면 -> 뒤에 0을 채운다.
- 길면 -> 100개까지만 쓰고 자른다.

In [ ]:
import torch.nn.functional as F

# --- [2] 패딩 (길이 맞추기) ---
seq_len = 100  # 모든 문장을 100단어로 통일

def pad_sequences(sequences, max_len):
    padded_list = []
    for seq in sequences:
        # 1. PyTorch 텐서로 변환 (Long 타입: 정수)
        seq_tensor = torch.tensor(seq, dtype=torch.long)
        
        # 2. 길이 조절
        if len(seq_tensor) < max_len:
            # 모자라면 뒤에 0 채우기 (Padding)
            # F.pad(입력, (왼쪽채움, 오른쪽채움), 값)
            padded = F.pad(seq_tensor, (0, max_len - len(seq_tensor)), value=0)
        else:
            # 넘치면 자르기 (Truncating)
            padded = seq_tensor[:max_len]
        
        padded_list.append(padded)
    
    # 리스트를 하나의 큰 텐서로 합치기 (Stack)
    return torch.stack(padded_list)

X_train_padded = pad_sequences(X_train, seq_len)
X_test_padded = pad_sequences(X_test, seq_len)

print(f"패딩된 데이터 형태: {X_train_padded.shape} -> (샘플15000개, 길이100)")

In [ ]:
# --- [3] 원-핫 인코딩 (선택 사항) ---
# 단어 번호(3)를 벡터([0, 0, 0, 1, 0, ...])로 바꾸는 작업
# 하지만 보통은 'Embedding Layer'를 쓰는데, 여기선 RNN 원리를 보려고 원-핫을 씀
# (메모리를 많이 먹으니 주의!)

X_train_onehot = F.one_hot(X_train_padded, num_classes=vocab_size).float()
X_test_onehot = F.one_hot(X_test_padded, num_classes=vocab_size).float()

print(f"원-핫 변환 후: {X_train_onehot.shape}")
# (15000개 문장, 100개 단어, 각 단어는 300칸짜리 벡터)

## 5. 모델 설계: 내 생애 첫 RNN 모델

구조는 간단합니다.
1. **RNN**: 문장을 읽고 감정(Hidden State)을 추출한다.
2. **Linear(FC)**: 추출된 감정을 0과 1 사이 점수로 바꾼다.

In [ ]:
import torch.nn as nn

class SimpleRNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        
        # 1. RNN 층: 문장을 읽는 눈과 뇌
        self.rnn = nn.RNN(
            input_size=input_dim,    # 단어(벡터)의 크기 (여기선 300)
            hidden_size=hidden_dim,  # 기억장치 크기 (뇌 용량, 여기선 8)
            batch_first=True         # (배치, 시간, 특징) 순서 유지
        )
        
        # 2. 출력 층: 긍정/부정 판단 (판사)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        # x: (배치, 100, 300) -> 문장들이 들어옴
        
        # RNN 실행
        # output: 모든 시점의 기록 (잘 안 씀)
        # hidden: 마지막 시점의 기억 (이게 중요! 문장 전체 요약본)
        output, hidden = self.rnn(x)
        
        # hidden은 (1, 배치, 은닉) 3차원이므로, (배치, 은닉) 2차원으로 펴줌
        last_memory = hidden.squeeze(0)
        
        # 긍정/부정 점수 계산
        score = self.fc(last_memory)
        return score

# 모델 생성
input_dim = vocab_size # 300
hidden_dim = 8         # 뇌 용량 (작게 잡음)
output_dim = 1         # 점수 하나 (긍정일 확률)

model = SimpleRNN(input_dim, hidden_dim, output_dim)
print("RNN 모델이 준비되었습니다!")

## 6. 학습 준비
이제 데이터를 모델에 넣고 학습(Training) 시킬 차례입니다.
- **Loss Function**: `BCEWithLogitsLoss` (이진 분류 문제니까)
- **Optimizer**: `Adam` (국룰)

In [ ]:
from torch.utils.data import TensorDataset, DataLoader
import torch.optim as optim

# 1. 라벨(정답)도 텐서로 변환
y_train_tensor = torch.tensor(y_train).float().unsqueeze(1) # (15000, 1)
y_test_tensor = torch.tensor(y_test).float().unsqueeze(1)

# 2. 데이터셋 만들기 (입력과 정답을 묶음)
train_dataset = TensorDataset(X_train_onehot, y_train_tensor)
test_dataset = TensorDataset(X_test_onehot, y_test_tensor)

# 3. 데이터 로더 (배치 단위로 꺼내줌)
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# 4. 설정
criterion = nn.BCEWithLogitsLoss() # 이진 분류 손실함수
optimizer = optim.Adam(model.parameters(), lr=0.001) # 최적화 도구

print("데이터셋과 학습 도구 준비 완료!")

In [ ]:
# --- [4] 실제 학습 돌리기 ---
num_epochs = 5

print("학습 시작...")
for epoch in range(num_epochs):
    model.train() # 학습 모드
    total_loss = 0
    
    for inputs, targets in train_loader:
        # 1. 예측
        outputs = model(inputs)
        
        # 2. 채점 (손실 계산)
        loss = criterion(outputs, targets)
        
        # 3. 모델 수정 (역전파)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    
    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}")

print("학습 끝! (Loss가 떨어지고 있다면 성공입니다)")

### 🌟 오늘의 결론
1. **RNN**은 이전 정보를 기억해서 다음을 예측한다.
2. 코드로 구현할 때는 `nn.RNN`을 쓰면 된다.
3. 하지만 RNN은 **길이가 길어지면 까먹는다(기울기 소멸)**.
4. 그래서 다음 시간에는 더 똑똑한 기억력을 가진 **LSTM**을 배울 것이다!